# Notes on grabbing all the DR12 BOSS spectra using igmspec

In [11]:
# imports
import pdb

from specdb.specdb import IgmSpec

## Load igmspec

In [2]:
igmsp = IgmSpec()

Using /u/xavier/local/Python/igmspec/DB/IGMspec_DB_v02.hdf5 for the catalog file
Using /u/xavier/local/Python/igmspec/DB/IGMspec_DB_v02.hdf5 for the DB file
Available surveys: [u'BOSS_DR12', u'HSTQSO', u'SDSS_DR7', u'KODIAQ_DR1', u'MUSoDLA', u'HD-LLS_DR1', u'2QZ', u'ESI_DLA', u'HDLA100', u'GGG', u'COS-Halos', u'HST_z2', u'COS-Dwarfs', u'XQ-100']
Database is igmspec
Created on 2016-Oct-25


## Examine main catalog

In [5]:
igmsp.qcat.cat[0:5]

sig_zem,flag_zem,flag_survey,zem,RA,DEC,IGM_ID,STYPE
float64,str10,float64,float64,float64,float64,int64,str3
-1.000,BOSS_PCA,1.0,2.308,0.0019,17.7737,0,QSO
0.002,BOSS_PCA,1.0,2.516,0.0028,14.9747,1,QSO
0.001,BOSS_PCA,1.0,1.629,0.0041,4.8298,2,QSO
0.000,BOSS_PCA,1.0,1.362,0.0053,-2.0333,3,QSO
0.001,BOSS_PCA,1.0,2.335,0.0057,-1.3250,4,QSO


## Use flag_survey [this should be an int not a float;  need to fix..]

### This grabs unique sources

In [6]:
boss = (igmsp.qcat.cat['flag_survey'] % 2) == 1

In [7]:
np.sum(boss)

302257

## Probably best to loop from here

In [10]:
igm_ids = igmsp.qcat.cat['IGM_ID'][boss]

In [27]:
# The following is pretty slow (one-by-one)
"""
for idx in igm_ids:  
    spec, meta = igmsp.idb.grab_spec('BOSS_DR12', idx)
    if isinstance(spec, list):  # Deals with the ones that have multiple spectra
        spec = spec[0]
    pdb.set_trace()
"""

"\nfor idx in igm_ids:  \n    spec, meta = igmsp.idb.grab_spec('BOSS_DR12', idx)\n    if isinstance(spec, list):  # Deals with the ones that have multiple spectra\n        spec = spec[0]\n    pdb.set_trace()\n"

In [15]:
# Grab 10000 at a time  (only a few times slower than 1)
ids = igm_ids[:10000]
spec, meta = igmsp.idb.grab_spec('BOSS_DR12', ids)

In [20]:
# The result is one XSpectrum1D object
spec

<XSpectrum1D: file=none, nspec=10003, select=0, wvmin=3604.96 Angstrom, wvmax=10289.6 Angstrom>

In [23]:
uni, uni_idx = np.unique(meta['IGM_ID'], return_index=True)
len(uni_idx)

10000

In [30]:
# Quick to loop on this
for idx in uni_idx:
    spec.select = idx
    if (idx % 1000) == 0:
        print(spec.wvmin)

3604.95654297 Angstrom
3604.95654297 Angstrom
3590.87353516 Angstrom
3566.15380859 Angstrom
3613.26611328 Angstrom
3566.97436523 Angstrom
3553.85693359 Angstrom
3590.0456543 Angstrom
3564.51171875 Angstrom
3604.95654297 Angstrom
